## 1.不含模型参数的自定义层

In [2]:
from mxnet import nd, gluon
from mxnet.gluon import nn

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
    
    def forward(self, x):
        return x - x.mean()

In [3]:
layer = CenteredLayer()
layer(nd.array([1, 2, 3]))


[-1.  0.  1.]
<NDArray 3 @cpu(0)>

In [4]:
net = nn.Sequential()
net.add(nn.Dense(128),
        CenteredLayer())

In [7]:
net.initialize()
y = net(nd.random.uniform(shape = (2, 20)))
y.mean().asscalar()

4.2200554e-10

## 2.含模型参数的自定义层

In [8]:
params = gluon.ParameterDict()
params.get('param2', shape = (2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [11]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape = (in_units, units))
        self.bias = self.params.get('bias', shape = (units, ))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

dense = MyDense(units = 3, in_units = 5)
dense.params

mydense1_ (
  Parameter mydense1_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense1_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [12]:
dense.initialize()
dense(nd.random.uniform(shape = (2, 5)))


[[0.12593447 0.04282258 0.02242133]
 [0.13126963 0.05341719 0.06172268]]
<NDArray 2x3 @cpu(0)>

In [13]:
net = nn.Sequential()
net.add(nn.Dense(8, in_units = 64),
        nn.Dense(1, in_units = 8))
net.initialize()
net(nd.random.uniform(shape = (2, 64)))


[[0.0337131 ]
 [0.03213731]]
<NDArray 2x1 @cpu(0)>